<a href="https://colab.research.google.com/github/vikpy/DLSem3/blob/master/Assignment_5_flowing_the_image_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imagenet for model building - classifier on any three classes (synsets) of cars 

In [3]:
#code part 1
from bs4 import BeautifulSoup
import numpy as np
import requests
import cv2
import PIL.Image
import urllib
import os
import pandas as pd


wnids = {
         
         'jeep' : 'n03594945',
         
         'race_car' : 'n04037443',

         'army_tank' : 'n04389033'

         }
image_net_path = 'http://www.image-net.org/api/text/imagenet.synset.geturls?wnid='
pages = {}
for wnid in wnids.keys():
  temp = requests.get(image_net_path + wnids[wnid])
  pages[wnid] = str(BeautifulSoup(temp.content, 'html.parser')).split('\r\n')

In [ ]:
pages["race_car"][:5]

['http://www.upmoon.com/up7/Mitsubishi%20Eclipse%20Street%20Racing%20Car.jpg',
 'http://www.hunternet.com.au/members/uploads/images/SkidPadFront.jpg',
 'http://farm1.static.flickr.com/202/463662585_60ea3e8381.jpg',
 'http://www.cardsunlimited.com/largeimage/RacingCar.jpg',
 'http://www.iceposter.com/thumbs/A11870_b.jpg']

We will be using Flow from Directory 
![alt text](http://upscfever.com/datasets/flow_from_directory.jpeg)

In [ ]:
# Making directories for the train set 
!mkdir /content/train 
!mkdir /content/train/race_car 
!mkdir /content/train/jeep 
!mkdir /content/train/army_tank 

# making directories for the test set
!mkdir /content/validation 
!mkdir /content/validation/race_car 
!mkdir /content/validation/jeep 
!mkdir /content/validation/army_tank 

# making directories for the test set 
!mkdir /content/test/ 
!mkdir /content/test/test 

In [ ]:
def url_to_image(url):
  import numpy as np
  import cv2
  import urllib
  resp = urllib.request.urlopen(url)
  image = np.asarray(bytearray(resp.read()), dtype="uint8")
  image = cv2.imdecode(image, cv2.IMREAD_COLOR)
 
	# return the image
  return image

In [ ]:
n_of_training_images=150#the number of training images to use

for class_of_vehicle in pages:
  for progress in range(n_of_training_images):
      if(progress%20==0):
          print(progress)
      if not pages[class_of_vehicle][progress] == None:
        try:
          I = url_to_image(pages[class_of_vehicle][progress])
          if (len(I.shape)) == 3: #check if the image has width, length and channels
            save_path = '/content/train/' + class_of_vehicle + '/img'+str(progress)+'.jpg'#create a name of each image
            cv2.imwrite(save_path,I)
        except:
          None

0
20
40
60
80
100
120
140
0
20
40
60
80
100
120
140
0
20
40
60
80
100
120
140


In [ ]:
for class_of_vehicle in pages:
  for progress in range(50):
      if(progress%20==0):
          print(progress)
      if not pages[class_of_vehicle][n_of_training_images + progress] == None:
        try:
          I = url_to_image(pages[class_of_vehicle][n_of_training_images + progress])
          if (len(I.shape)) == 3: #check if the image has width, length and channels
            save_path = '/content/validation/' + class_of_vehicle + '/img'+str(n_of_training_images + progress)+'.jpg'#create a name of each image
            cv2.imwrite(save_path,I)
        except:
          None

0
20
40
0
20
40
0
20
40


In [ ]:
for class_of_vehicle in pages:
  for progress in range(50):
      if(progress%20==0):
          print(progress)
      if not pages[class_of_vehicle][n_of_training_images + progress + 50] == None:
        try:
          I = url_to_image(pages[class_of_vehicle][n_of_training_images + progress + 50])
          if (len(I.shape)) == 3: #check if the image has width, length and channels
            save_path = '/content/test/test/'+ str(class_of_vehicle) +  '_' +str(n_of_training_images + progress + 50)+'.jpg'#create a name of each image
            cv2.imwrite(save_path,I)
        except:
          None

0
20
40
0
20
40
0
20
40


In [ ]:
# Path to the train folder
original_test = '/content/test/test'
 
filenames = os.listdir(original_test)
categories = []
for filename in filenames:
    category = filename.split('_')[0]
    if category == 'ships':
        categories.append('ships')
    else:
        categories.append('bikes')

data_test = pd.DataFrame({'filename':filenames,'label':categories})

data_test.to_csv("original_test.csv",index=False)